In [1]:
import timeit
import numpy as np
import matplotlib.pyplot as plt
import pickle
from time import time

In [2]:
def init_kernel_bias(num_inp_channels, kernel_size, num_kernels,mean=0,std=0.01):
    shape = [num_inp_channels, kernel_size, kernel_size, num_kernels]
    weights = std*np.random.randn(*shape) + mean
    # weights/=np.sqrt(num_inp_channels)
    bias = std*np.random.randn(1,num_kernels) + mean
    return weights.astype(np.float32), bias.astype(np.float32)

In [5]:
def conv2d(self,inp,kernels,biases,stride=[1,1],padding=0):     #padding=(ksz-1)/2 for same shape in stride 1
    #inp[batches,row,col,d],kernels(d,ksz,ksz,num_ker),biases[1,num_ker],stride[row,col]
    inp=inp.transpose(0,3,1,2)  #inp[batches,d,row,col]
    output=[]
    ksz=kernels.shape[1]
    num_ker=kernels.shape[3]
    if not padding:                         #take care of padding in backprop too
        padding=(ksz-1)//2                  #currently don't give 'even' ksz
    out_row,out_col=((inp.shape[2]-ksz+2*padding)//stride[0]+1),((inp.shape[3]-ksz+2*padding)//stride[1]+1)
    batches,d,row,col=inp.shape
    row+=2*padding
    col+=2*padding
    padded=np.zeros((batches,d,row,col))
    padded[:,:,padding:-padding,padding:-padding]=inp
    # Take all windows into a matrix       # TO-DO: Cache up these calculations
    window=(np.arange(ksz)[:,None]*row+np.arange(ksz)).ravel()+np.arange(d)[:,None]*row*col
    slider=(np.arange(out_row*stride[0])[:,None]*row+np.arange(out_col*stride[1]))
    ind = window.ravel()+slider[::stride[0],::stride[1]].ravel()[:,None]
    kern = kernels.reshape(-1,num_ker)
    for img in padded:      #img[d,row,col]     #TODO: MAKE THIS FOR LOOP INTO SINGLE NUMPY OPERATION
        # windows(out_row*out_col, ksz*ksz*d) . kernels(d*ksz*ksz,num_ker)
        out=(np.dot(np.take(img, ind), kern))
        out=(out+biases).reshape(out_row,out_col,num_ker)
        output.append(out)
#     output=np.array([(np.dot(np.take(i,ind),kern)+biases) for i in padded]).reshape(batches,out_row,out_col,num_ker)
#     bind= np.arange(batches)[:,None]*d*row*col+ind.ravel()		#for batches
#     output=(np.dot(np.take(padded, bind).reshape(-1,d*ksz*ksz), kern)+biases).reshape(batches,out_row,out_col,num_ker)
    return np.array(output) #output[batches,out_row,out_col,num_ker]

In [ ]:
%timeit conv0=conv2d(1,inp,w0,b0)

In [ ]:
%timeit conv0=nn.conv2d(inp,w0,b0)

In [ ]:
conv0=conv2d(1,inp,w0,b0)

In [ ]:
%timeit d_X_inp,d_w0,d_b0=nn.conv2d_back(errors=conv0,inp=inp,kernels=w0,biases=b0)

In [ ]:
conv0.shape

In [4]:
inp=np.random.randint(0,9,(1,4,4,1)).astype(np.float32)
w0,b0=init_kernel_bias(num_inp_channels=1,kernel_size=3,num_kernels=1)
w0.shape

(1, 3, 3, 1)

In [14]:
inp=np.random.randint(0,9,(128,32,32,64)).astype(np.float32)
w0,b0=init_kernel_bias(num_inp_channels=64,kernel_size=3,num_kernels=32)
w0.shape

(1, 16, 16, 32)

In [6]:
dilation=[1,1]

In [7]:
#inp[batches,row,col,d],w0(d,ksz,ksz,num_ker),b0[1,num_ker],stride[row,col]
padding=1
stride=[2,2]
ipp=inp.transpose(0,3,1,2)  #ipp[batches,d,row,col]
output=[]
ksz=w0.shape[1]
num_ker=w0.shape[3]
if not padding: #take care of padding in backprop too
    padding=(ksz-1)//2  #currently don't give 'even' ksz
out_row,out_col=((ipp.shape[2]-ksz+2*padding-(ksz-1)*(dilation[0]-1))//stride[0]+1),((ipp.shape[3]-ksz+2*padding-(ksz-1)*(dilation[1]-1))//stride[1]+1)
batches,d,row,col=ipp.shape
row+=2*padding
col+=2*padding
padded=np.zeros((batches,d,row,col),dtype=np.float32)
padded[:,:,padding:-padding,padding:-padding]=ipp

In [9]:
%timeit padded[:,:,padding:-padding:1,padding:-padding:1]=ipp

39.1 ms ± 244 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [10]:
img=padded[0]
row,col,d

(66, 66, 32)

In [11]:
out_row=2
out_col=2

In [12]:
dksz=ksz+(ksz-1)*(dilation[0]-1)

In [13]:
# %%timeit
window=(np.arange(dksz,step=dilation[0])[:,None]*row+np.arange(dksz,step=dilation[1])).ravel()+np.arange(d)[:,None]*row*col
slider=(np.arange(out_row*stride[0])[:,None]*row+np.arange(out_col*stride[1]))
ind = window.ravel()+slider[::stride[0],::stride[1]].ravel()[:,None]
# bind= np.arange(batches)[:,None]*d*row*col+ind.ravel()
kern = w0.reshape(-1,num_ker)
# output=(np.dot(np.take(padded, bind).reshape(-1,d*ksz*ksz), kern)).reshape(batches,out_row,out_col,num_ker)

In [14]:
img.shape,row,col,out_row,out_col,ksz

((32, 66, 66), 66, 66, 2, 2, 3)

In [15]:
window

array([[     0,      1,      2,     66,     67,     68,    132,    133,
           134],
       [  4356,   4357,   4358,   4422,   4423,   4424,   4488,   4489,
          4490],
       [  8712,   8713,   8714,   8778,   8779,   8780,   8844,   8845,
          8846],
       [ 13068,  13069,  13070,  13134,  13135,  13136,  13200,  13201,
         13202],
       [ 17424,  17425,  17426,  17490,  17491,  17492,  17556,  17557,
         17558],
       [ 21780,  21781,  21782,  21846,  21847,  21848,  21912,  21913,
         21914],
       [ 26136,  26137,  26138,  26202,  26203,  26204,  26268,  26269,
         26270],
       [ 30492,  30493,  30494,  30558,  30559,  30560,  30624,  30625,
         30626],
       [ 34848,  34849,  34850,  34914,  34915,  34916,  34980,  34981,
         34982],
       [ 39204,  39205,  39206,  39270,  39271,  39272,  39336,  39337,
         39338],
       [ 43560,  43561,  43562,  43626,  43627,  43628,  43692,  43693,
         43694],
       [ 47916,  4791

In [16]:
slider

array([[  0,   1,   2,   3],
       [ 66,  67,  68,  69],
       [132, 133, 134, 135],
       [198, 199, 200, 201]])

In [28]:
ind

array([[ 0,  1,  2,  6,  7,  8, 12, 13, 14],
       [ 2,  3,  4,  8,  9, 10, 14, 15, 16],
       [12, 13, 14, 18, 19, 20, 24, 25, 26],
       [14, 15, 16, 20, 21, 22, 26, 27, 28]])

In [17]:
ind.size*batches,ind.shape

(147456, (4, 288))

In [18]:
# padded=np.arange(padded.size,dtype=np.float32).reshape(padded.shape)

In [19]:
padded.shape

(128, 32, 66, 66)

In [ ]:
coled=np.empty(ind.size*batches,dtype=np.float32).reshape(-1,d*ksz*ksz)

In [69]:
batches,out_row,out_col

(2, 3, 3)

In [70]:
coled.shape

(18, 4)

In [ ]:
%%time
ctake.take(c_void_p(padded.ctypes.data),c_void_p(ind.ctypes.data),c_void_p(coled.ctypes.data),c_int(1),c_int(padded[0].size),c_int(ind.size),c_int(4))

In [72]:
a=conv2d(32,kernel_size=3,stride=[2,2],input_shape=(32,32,3))

In [55]:
a.shape

(None, 4, 4, 1)

In [76]:
%%time
output=np.empty((batches,out_row*out_col,num_ker))
for i,img in enumerate(padded):      #img[d,row,col]
    # windows(out_row*out_col, ksz*ksz*d) . kernels(d*ksz*ksz,num_ker)
    output[i]=np.dot(img.take(ind), kern)+b0
ans=output.reshape(batches,out_row,out_col,num_ker)

CPU times: user 386 µs, sys: 0 ns, total: 386 µs
Wall time: 309 µs


In [77]:
output.shape,ans.shape

((2, 9, 1), (2, 3, 3, 1))

In [25]:
%%time
output=np.empty((batches,out_row*out_col,num_ker))
for i,img in enumerate(padded):      #img[d,row,col]
    # windows(out_row*out_col, ksz*ksz*d) . kernels(d*ksz*ksz,num_ker)
    output[i]=img.take(ind).dot(kern)
if b0 is not 0:
    output+=b0
ans=output.reshape(batches,out_row,out_col,num_ker)

CPU times: user 106 µs, sys: 0 ns, total: 106 µs
Wall time: 80.3 µs


In [28]:
%%timeit
output=np.empty((batches,out_row*out_col,num_ker))
for i,img in enumerate(padded):      #img[d,row,col]
    # windows(out_row*out_col, ksz*ksz*d) . kernels(d*ksz*ksz,num_ker)
    output[i]=np.dot(img.take(ind), kern)
output+=b0
ans2=output.reshape(batches,out_row,out_col,num_ker)

1.24 s ± 11 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [98]:
%timeit np.array([(np.dot(np.take(i,ind),kern)) for i in padded]).reshape(batches,out_row,out_col,num_ker)

1.39 s ± 80.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [95]:
%timeit (np.dot(np.take(padded, bind).reshape(-1,d*ksz*ksz), kern)).reshape(batches,out_row,out_col,num_ker)

928 ms ± 54.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [96]:
%timeit (np.dot(padded.take(bind).reshape(-1,d*ksz*ksz), kern)).reshape(batches,out_row,out_col,num_ker)

931 ms ± 20.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [23]:
np.take(img,ind).shape,kern.shape

((4096, 288), (288, 128))

In [24]:
(np.dot(np.take(img,ind), kern)).shape

(4096, 128)

In [26]:
ans=(np.dot(np.take(padded, bind).reshape(-1,d*ksz*ksz), kern)+b0).reshape(batches,out_row,out_col,num_ker)

In [27]:
ans.shape

(128, 64, 64, 128)

In [1]:
from nnet.layers import Layer,conv2d

Seed: 424


In [2]:
import cupy as cp
import numpy as np

In [3]:
def init_kernel_bias(num_inp_channels, kernel_size, num_kernels,mean=0,std=0.01):
    shape = [num_inp_channels, kernel_size, kernel_size, num_kernels]
    weights = std*np.random.randn(*shape) + mean
    # weights/=np.sqrt(num_inp_channels)
    bias = std*np.random.randn(1,num_kernels) + mean
    return weights.astype(np.float32), bias.astype(np.float32)

In [4]:
w0,b0=init_kernel_bias(num_inp_channels=32,kernel_size=3,num_kernels=64)

In [5]:
inp=np.random.randn(128,60,60,32).astype(np.float32)

In [6]:
inp.shape,w0.shape

((128, 60, 60, 32), (32, 3, 3, 64))

In [7]:
inpd=cp.asarray(inp.transpose(0,3,1,2))
w0d=cp.asarray(w0.transpose(3,0,1,2))

In [8]:
nk, c, kh, kw = w0d.shape
bt, c, h, w = inpd.shape
sy,sx = (1,1)
ph,pw = (1,1)
dy,dx = (1,1)
out_h,out_w = (h,w)
col = cp.empty((bt, c, kh, kw, out_h, out_w), dtype=inpd.dtype)
colkern = cp.ElementwiseKernel(
    'raw T inpd, int32 h, int32 w, int32 out_h, int32 out_w,'
    'int32 kh, int32 kw, int32 sy, int32 sx, int32 ph, int32 pw,'
    'int32 dy, int32 dx',
    'T col',
    '''
       int c0 = i / (kh * kw * out_h * out_w);   // select channel
       int ky = i / (kw * out_h * out_w) % kh;   // select kernel y
       int kx = i / (out_h * out_w) % kw;        // select kernel x
       int out_y = i / out_w % out_h;            // select output y
       int out_x = i % out_w;                    // select output x
       int in_y = ky * dy + out_y * sy - ph;
       int in_x = kx * dx + out_x * sx - pw;
       if (in_y >= 0 && in_y < h && in_x >= 0 && in_x < w) {    // if in image bounds
         col = inpd[w * (in_y + h * c0) + in_x]; // choose pixel
       } else {
         col = 0;                                // pad with 0
       }
    ''',
    'im2col')

In [48]:
flipped=w0d[:,::-1,::-1,:].transpose(3,1,2,0)

In [10]:
col.shape,w0d.shape

((128, 32, 3, 3, 60, 60), (64, 32, 3, 3))

In [15]:
%%time
global col
col=colkern(inpd.reduced_view(),
              h, w, out_h, out_w, kh, kw, sy, sx, ph, pw, dy, dx, col)
outd=cp.tensordot(col, w0d, ((1, 2, 3), (1, 2, 3)))
cp.cuda.Stream.null.synchronize()

CPU times: user 462 ms, sys: 345 µs, total: 462 ms
Wall time: 461 ms


In [16]:
outd.shape

(128, 60, 60, 64)

In [17]:
col=col.transpose(0,4,5,1,2,3)
w0d=w0d.transpose(1,2,3,0)

In [18]:
col.shape,w0d.shape

((128, 60, 60, 32, 3, 3), (32, 3, 3, 64))

In [19]:
c=conv2d(input_shape=(60,60,32),kernels=w0,biases=0)

In [23]:
%%time
out=c.forward(inp)

CPU times: user 1.16 s, sys: 24.4 ms, total: 1.19 s
Wall time: 358 ms


In [26]:
outd.shape,out.shape

((128, 60, 60, 64), (128, 60, 60, 64))

In [27]:
np.allclose(out,outd.get(),atol=1e-06/2)

True

In [28]:
(128,60,60,32,3,3),c.kernels.shape

((128, 60, 60, 32, 3, 3), (32, 3, 3, 64))

In [30]:
col=cp.empty(c.coled.shape,dtype=cp.float32)
dpadded=cp.asarray(c.padded)
dind=cp.asarray(c.ind.astype(np.int32,copy=False))

In [31]:
dbind=cp.broadcast_to(dind,(128,*dind.shape))

In [32]:
col.shape,c.ind.shape,dbind.shape

((460800, 288), (3600, 288), (128, 3600, 288))

In [33]:
dkern=cp.asarray(c.kern)

In [34]:
svimkern=cp.ElementwiseKernel(
    'raw int32 bind,raw T padded,int32 indsz,int32 imgsz',
    'T col',
    '''
    int n = i/indsz;
    col = padded[bind[i]+n*imgsz];
    ''',
    'assign')

In [38]:
%%time
svimkern(dbind,dpadded,dind.size,dpadded[0].size,col)
dout=col.dot(dkern)
cp.cuda.Stream.null.synchronize()

CPU times: user 546 ms, sys: 541 µs, total: 547 ms
Wall time: 547 ms


In [39]:
(dout.ravel()==outd.ravel()).all()

array(True)

In [40]:
col.shape,c.kern.shape

((460800, 288), (288, 64))